# Blog
- https://lsjsj92.tistory.com/657

위 블로그에서 설명한 코드입니다.

# Data
- https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset

In [1]:
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [371]:
import requests

import pandas as pd
import numpy as np
import copy
import json

from ast import literal_eval

import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
from transformers import OwlViTProcessor, OwlViTForObjectDetection
from transformers import pipeline
from transformers import GPT2TokenizerFast
from PIL import Image

import pickle



In [3]:
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict

import sklearn.datasets as datasets
import sklearn.manifold as manifold

In [4]:
import openai
import os
import sys
from dotenv import load_dotenv

load_dotenv()    
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
cur_os = sys.platform

In [6]:
model_path = f"D:/github" if cur_os.startswith('win') else None

## 데이터셋

In [7]:
movies_metadata = pd.read_csv('./movie_meta/movies_metadata.csv', sep=",", dtype=str)
print(movies_metadata.shape)
movies_metadata.head()

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


In [8]:
movies_metadata = movies_metadata[['id', 'genres', 'title', 'overview', 'release_date']]
movies_metadata.head()

,id,genres,title,overview,release_date
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,A family wedding reignites the ancient feud be...,1995-12-22
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",1995-12-22
4,11862,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,Just when George Banks has recovered from his ...,1995-02-10


In [9]:
movies_metadata['genres'] = movies_metadata['genres'].apply(literal_eval)
movies_metadata.head()

,id,genres,title,overview,release_date
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,A family wedding reignites the ancient feud be...,1995-12-22
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",1995-12-22
4,11862,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,Just when George Banks has recovered from his ...,1995-02-10


## 사용할 컬럼 설정

In [10]:
def get_genre(x):
    names = [i['name'] for i in x]
    if len(names) > 3:
        names = names[:3]
    return " ".join(names)

In [11]:
movies_metadata['genres'] = movies_metadata['genres'].apply(lambda x : get_genre(x))

In [12]:
movies_metadata.head()

,id,genres,title,overview,release_date
0,862,Animation Comedy Family,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30
1,8844,Adventure Fantasy Family,Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15
2,15602,Romance Comedy,Grumpier Old Men,A family wedding reignites the ancient feud be...,1995-12-22
3,31357,Comedy Drama Romance,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",1995-12-22
4,11862,Comedy,Father of the Bride Part II,Just when George Banks has recovered from his ...,1995-02-10


In [13]:
movies_metadata = movies_metadata[movies_metadata['overview'].notnull()]
movies_metadata = movies_metadata[movies_metadata['title'].notnull()]
movies_metadata.isna().sum()

index            0
id               0
genres           0
title            0
overview         0
release_date    71
dtype: int64

In [14]:
movies_metadata['feature'] = movies_metadata['genres'] + " / " + movies_metadata['title'] + " / " + movies_metadata['overview']

# HuggingFace embedding

In [18]:
if cur_os.startswith('win'):
    model = SentenceTransformer(f'{model_path}/distiluse-base-multilingual-cased-v2')    
else:
    model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")

model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [24]:
movies_metadata['hf_embeddings'] = movies_metadata['feature'].apply(lambda x : model.encode(x))
movies_metadata.shape

(44506, 10)

In [48]:
print(movies_metadata.shape)
movies_metadata.head()

(44506, 9)


,id,genres,title,overview,release_date,feature,text_len,feature_len,hf_embeddings
0,862,Animation Comedy Family,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Animation Comedy Family / Toy Story / Led by W...,303,341,"[-0.047183443, -0.02021129, 0.096098304, -0.05..."
1,8844,Adventure Fantasy Family,Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15,Adventure Fantasy Family / Jumanji / When sibl...,395,432,"[0.0013290617, -0.0071765357, 0.048141554, -0...."
2,15602,Romance Comedy,Grumpier Old Men,A family wedding reignites the ancient feud be...,1995-12-22,Romance Comedy / Grumpier Old Men / A family w...,327,363,"[-0.06425337, -0.008573138, -0.10116352, -0.00..."
3,31357,Comedy Drama Romance,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Comedy Drama Romance / Waiting to Exhale / Che...,270,313,"[-0.032623984, -0.03347266, 0.02557934, -0.033..."
4,11862,Comedy,Father of the Bride Part II,Just when George Banks has recovered from his ...,1995-02-10,Comedy / Father of the Bride Part II / Just wh...,318,357,"[0.03181218, -0.0038158156, 0.02099277, 0.0031..."


In [49]:
movies_metadata.to_csv('./movie_meta/movies_metadata_em.csv')

# OpenAI Embedding

In [26]:
openai_embedding_model = "text-embedding-ada-002"

In [27]:
def get_doc_embedding(text: str) -> List[float]:
    return get_embedding(text, openai_embedding_model)

In [28]:
def get_embedding(text: str, model: str) -> List[float]:
    result = openai.Embedding.create(
      model=model,
      input=text)
    return result["data"][0]["embedding"]

In [29]:
# movies_metadata['openai_embeddings'] = movies_metadata['feature'].apply(lambda x : get_embedding(x, openai_embedding_model))

# Query와 비슷한 앱

In [30]:
top_k = 5

In [671]:
def get_query_sim_top_k(query, model, df, top_k):
    query_encode = model.encode(query)
    cos_scores = util.pytorch_cos_sim(query_encode, df['hf_embeddings'])[0]
    top_results = torch.topk(cos_scores, k=top_k)
    return top_results

In [32]:
query = "Are there any documentary films?"
top_result = get_query_sim_top_k(query, model, movies_metadata)

/Users/leesoojin/opt/anaconda3/envs/openai/lib/python3.8/site-packages/sentence_transformers/util.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  b = torch.tensor(b)


In [33]:
top_result

torch.return_types.topk(
values=tensor([0.5390, 0.5117, 0.5093, 0.5067, 0.4992]),
indices=tensor([24020, 10124, 22428, 35263, 22273]))

In [34]:
movies_metadata.iloc[top_result[1].numpy(), :][['title', 'overview', 'genres']]

,title,overview,genres
24020,The 50 Worst Movies Ever Made,There are some movies that are so bad they're ...,Documentary
10124,Trekkies 2,sequel to the 1997 documentary film Trekkies.,Documentary
22428,The Spanish Earth,A propaganda film made during the Spanish Civi...,War Documentary
35263,Tomorrow,Documentary film about global warming.,Documentary
22273,I Know That Voice,A documentary about voice-over actors.,Documentary


In [35]:
query = "Are there any movies about natural disasters?"
top_result = get_query_sim_top_k(query, model, movies_metadata)
movies_metadata.iloc[top_result[1].numpy(), :][['title', 'overview', 'genres']]

,title,overview,genres
38573,Catastrophe,A film cataloguing some of the world's largest...,Thriller Documentary
11441,When the Levees Broke: A Requiem in Four Acts,"In August 2005, the American city of New Orlea...",Documentary
2404,Earthquake,Earthquake is a 1974 American disaster film th...,Action Drama Thriller
35263,Tomorrow,Documentary film about global warming.,Documentary
41943,Disaster!,"A spoof of disaster films, an asteroid is comi...",Action Animation Comedy


In [36]:
query = "Are there any movies about heros?"
top_result = get_query_sim_top_k(query, model, movies_metadata)
movies_metadata.iloc[top_result[1].numpy(), :][['title', 'overview', 'genres']]

,title,overview,genres
36936,The Flying Man,"A new superhero is coming, only this time it's...",Action Mystery Science Fiction
30101,"Up, Up, and Away",A boy is the only family member without superp...,Action Family TV Movie
24646,The Four Feathers,They made him a hero by branding him a coward ...,TV Movie Adventure Drama
12672,Hancock,Hancock is a down-and-out superhero who's forc...,Fantasy Action
4216,Too Late the Hero,A WWII film set on a Pacific island. Japanese ...,Drama Action War


https://www.imdb.com/title/tt0211174/?ref_=fn_al_tt_1

# ChatGPT 이용

- 2개의 chatgpt 이용
    - 1개는 이 질문의 의도를 파악하는 것. 설명을 원하는 것인지, 추천을 해달라는 것인지
    - 각 분류에 따라 문구가 달라짐
        - 질문의 의도면 가장 유사한 텍스트를 가져와서 설명해주는 것
        - 추천이면, cossim topk를 가져와서 출력하도록

In [37]:
def print_msg(msg):
    completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=msg
                    )
    return completion['choices'][0]['message']['content']

## Prompt test

In [38]:
messages = [
    {"role": "system", "content": "You are a helpful assistant who understands the intent of the user's question."},
    {"role": "user", "content": "Which category does the sentence below belong to: 'description', 'recommended'? Show only categories. \n context: tell me about instagram \n A:"}
]


print_msg(messages)

'description'

In [39]:
messages = [
    {"role": "system", "content": "You are a helpful assistant who understands the intent of the user's question."},
    {"role": "user", "content": "Which category does the sentence below belong to: 'description', 'recommend'? Show only categories. \n context: What apps are similar to Instagram? \n A:"}
]


print_msg(messages)



'recommend'

In [40]:
messages = [
    {"role": "system", "content": "You are a helpful assistant who understands the intent of the user's question."},
    {"role": "user", "content": "Which category does the sentence below belong to: 'description', 'recommend'? Show only categories. \n context: Recommend apps similar to Instagram. \n A:"}
]


print_msg(messages)

'recommend'

In [41]:
messages = [
    {"role": "system", "content": "You are a helpful assistant who recommend contents."},
    {"role": "user", "content": "Simply repeat the provided context and put a sentence in front of the context. \n context: Recommend apps similar to Instagram."}
]


print_msg(messages)

'Here are some apps similar to Instagram that you might want to check out:'

In [42]:
messages = [
    {"role": "system", "content": "You are a helpful assistant who recommend contents."},
    {"role": "user", "content": "Simplify the sentences for recommending services \n context: Recommend apps similar to Instagram."}
]


print_msg(messages)

'Here are some apps like Instagram that you might find helpful!'

In [43]:
messages = [
    {"role": "system", "content": "You are a helpful assistant who kindly answers."},
    {"role": "user", "content": "Please write a simple greeting starting with 'of course' to explain the item to the user."}
]


print_msg(messages)

"Of course! I'd be happy to explain the item to you."

In [44]:
messages = [
    {"role": "system", "content": "You are a helpful assistant who recommend contents based on user question."},
    {"role": "user", "content": "Write 1 sentence of a simple greeting that starts with 'Of course!' to recommend items to users."}
]


print_msg(messages)

"Of course! I'd be happy to recommend some great items for you."

In [103]:
movies_metadata.iloc[top_result[1].numpy(), :][['title', 'overview', 'genres']]

,title,overview,genres
36936,The Flying Man,"A new superhero is coming, only this time it's...",Action Mystery Science Fiction
30101,"Up, Up, and Away",A boy is the only family member without superp...,Action Family TV Movie
24646,The Four Feathers,They made him a hero by branding him a coward ...,TV Movie Adventure Drama
12672,Hancock,Hancock is a down-and-out superhero who's forc...,Fantasy Action
4216,Too Late the Hero,A WWII film set on a Pacific island. Japanese ...,Drama Action War


## 필요한 Prompt 설정

- 추천인가? 설명인가? 의도 파악인가?

In [815]:
msg_prompt = {
    'recom' : {
                'system' : "You are a helpful assistant who recommend movie based on user question.", 
                'user' : "Write 1 sentence of a simple greeting that starts with 'Of course!' to recommend movie items to users.", 
              },
    'desc' : {
                'system' : "You are a helpful assistant who kindly answers.", 
                'user' : "Please write a simple greeting starting with 'of course' to explain the item to the user.", 
              },
    'intent' : {
                'system' : "You are a helpful assistant who understands the intent of the user's question.",
                'user' : "Which category does the sentence below belong to: 'description', 'recommended', 'search'? Show only categories. \n context:"
                }
}

In [856]:
msg_prompt

{'recom': {'system': 'You are a helpful assistant who recommend movie based on user question.',
  'user': "Write 1 sentence of a simple greeting that starts with 'Of course!' to recommend movie items to users."},
 'desc': {'system': 'You are a helpful assistant who kindly answers.',
  'user': "Please write a simple greeting starting with 'of course' to explain the item to the user."},
 'intent': {'system': "You are a helpful assistant who understands the intent of the user's question.",
  'user': "Which category does the sentence below belong to: 'description', 'recommended', 'search'? Show only categories. \n context:"}}

In [870]:
user_msg_history = []

In [871]:
def get_chatgpt_msg(msg):
    completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=msg
                    )
    return completion['choices'][0]['message']['content']

In [872]:
def set_prompt(intent, query, msg_prompt_init, model):
    '''prompt 형태를 만들어주는 함수'''
    m = dict()
    # 검색 또는 추천이면
    if ('recom' in intent) or ('search' in intent):
        msg = msg_prompt_init['recom'] # 시스템 메세지를 가지고오고
    # 설명문이면
    elif 'desc' in intent:
        msg = msg_prompt_init['desc'] # 시스템 메세지를 가지고오고
    # intent 파악
    else:
        msg = msg_prompt_init['intent']
        msg['user'] += f' {query} \n A:'
    for k, v in msg.items():
        m['role'], m['content'] = k, v
    return [m]

In [872]:
def user_interact(query, model, msg_prompt_init):
    # 1. 사용자의 의도를 파악
    user_intent = set_prompt('intent', query, msg_prompt_init, None)
    user_intent = get_chatgpt_msg(user_intent).lower()
    print("user_intent : ", user_intent)
    
    # 2. 사용자의 쿼리에 따라 prompt 생성    
    intent_data = set_prompt(user_intent, query, msg_prompt_init, model)
    intent_data_msg = get_chatgpt_msg(intent_data).replace("\n", "").strip()
    print("intent_data_msg : ", intent_data_msg)
    
    # 3-1. 추천 또는 검색이면
    if ('recom' in user_intent) or ('search' in user_intent):
        recom_msg = str()
        # 기존에 메세지가 있으면 쿼리로 대체
        if (len(user_msg_history) > 0 ) and (user_msg_history[-1]['role'] == 'assistant'):
            query = user_msg_history[-1]['content']['feature']
        # 유사 아이템 가져오기
        #top_result = get_query_sim_top_k(query, model, movies_metadata, top_k=1 if 'recom' in user_intent else 3) # 추천 개수 설정하려면!
        top_result = get_query_sim_top_k(query, model, movies_metadata, top_k=3)
        #print("top_result : ", top_result)
        # 검색이면, 자기 자신의 컨텐츠는 제외
        top_index = top_result[1].numpy() if 'recom' in user_intent else top_result[1].numpy()[1:]
        #print("top_index : ", top_index)
        # 장르, 제목, overview를 가져와서 출력
        r_set_d = movies_metadata.iloc[top_index, :][['genres', 'title', 'overview']]
        r_set_d = json.loads(r_set_d.to_json(orient="records"))
        for r in r_set_d:
            for _, v in r.items():
                recom_msg += f"{v} \n"
            recom_msg += "\n"
        user_msg_history.append({'role' : 'assistant', 'content' : f"{intent_data_msg} {str(recom_msg)}"})
        print(f"\n recom data : {intent_data_msg} {str(recom_msg)}")
    # 3-2. 설명이면
    elif 'desc' in user_intent:
        # 이전 메세지에 따라서 설명을 가져와야 하기 때문에 이전 메세지 컨텐츠를 가져옴
        top_result = get_query_sim_top_k(user_msg_history[-1]['content'], model, movies_metadata, top_k=1)
        # feature가 상세 설명이라고 가정하고 해당 컬럼의 값을 가져와 출력
        r_set_d = movies_metadata.iloc[top_result[1].numpy(), :][['feature']]
        r_set_d = json.loads(r_set_d.to_json(orient="records"))[0]
        user_msg_history.append({'role' : 'assistant', 'content' : r_set_d})
        print(f"\n describe : {intent_data_msg} {r_set_d}")

## 위의 user_interact 함수처럼 구성하지 말고 


In [872]:
'''

import openai

openai.api_key = "YOUR_API_KEY" # supply your API key however you choose

message = {"role":"user", "content": input("This is the beginning of your chat with AI. [To exit, send \"###\".]\n\nYou:")};

conversation = [{"role": "system", "content": "DIRECTIVE_FOR_gpt-3.5-turbo"}]

while(message["content"]!="###"):
    conversation.append(message)
    completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=conversation) 
    message["content"] = input(f"Assistant: {completion.choices[0].message.content} \nYou:")
    print()
    conversation.append(completion.choices[0].message)
    
'''

## 위처럼 하는 것이 깔끔할 수 있습니다.

## 쿼리에 따른 추천 프로세스 실행

In [874]:
query = "Please recommend a movie similar to a marvel heros movie."
user_interact(query, model, copy.deepcopy(msg_prompt))

user_intent :  recommended
intent_data_msg :  Of course! Here are some top-rated movie items that you might enjoy.

 recom data : Of course! Here are some top-rated movie items that you might enjoy. 

X-Men 
Two mutants, Rogue and Wolverine, come to a private academy for their kind whose resident superhero team, the X-Men, must oppose a terrorist organization with similar powers. 




In [876]:
query = "Can you describe on the above?"
user_interact(query, model, copy.deepcopy(msg_prompt))

user_intent :  description
intent_data_msg :  Of course! Let me explain what this item is and how it works.

 describe : Of course! Let me explain what this item is and how it works. {'feature': 'Adventure Action Science Fiction / X-Men / Two mutants, Rogue and Wolverine, come to a private academy for their kind whose resident superhero team, the X-Men, must oppose a terrorist organization with similar powers.'}


In [877]:
user_msg_history

[{'role': 'assistant',
  'content': 'Of course! Here are some top-rated movie items that you might enjoy. \n\nX-Men \nTwo mutants, Rogue and Wolverine, come to a private academy for their kind whose resident superhero team, the X-Men, must oppose a terrorist organization with similar powers. \n\n'},
 {'role': 'assistant',
  'content': {'feature': 'Adventure Action Science Fiction / X-Men / Two mutants, Rogue and Wolverine, come to a private academy for their kind whose resident superhero team, the X-Men, must oppose a terrorist organization with similar powers.'}}]

In [878]:
query = "Are there other movies that are similar to the ones above?"
user_interact(query, model, copy.deepcopy(msg_prompt))

user_intent :  'search'
intent_data_msg :  Of course! We have a great selection of movie items that will fit your every need.

 recom data : Of course! We have a great selection of movie items that will fit your every need. 
X-Men: Days of Future Past 
The ultimate X-Men ensemble fights a war for the survival of the species across two time periods as they join forces with their younger selves in an epic battle that must change the past – to save our future. 




# 사용자 태그

In [717]:
user_perfer_tag = ['computer','tech','science']

In [718]:
top_result = get_query_sim_top_k(' '.join(user_perfer_tag), model, movies_metadata, top_k=3 )

In [719]:
movies_metadata.iloc[top_result[1].numpy(), :][['title', 'genres', 'overview']]

,title,genres,overview
23040,Transcendence,Thriller Science Fiction Drama,Two leading computer scientists work toward th...
30222,Debug,Horror Science Fiction,Six young computer hackers sent to work on a d...
996,The Lawnmower Man,Horror Thriller Science Fiction,A simple man is turned into a genius through t...


In [723]:
movies_metadata[movies_metadata['title'] == 'X-Men: Days of Future Past']

,id,genres,title,overview,release_date,feature,text_len,feature_len,hf_embeddings
23165,127585,Action Adventure Fantasy,X-Men: Days of Future Past,The ultimate X-Men ensemble fights a war for t...,2014-05-15,Action Adventure Fantasy / X-Men: Days of Futu...,208,264,"[-0.047461677, 0.002811962, -0.010485606, -0.0..."
